In [6]:
import numpy as np
import time
from scipy.stats import lognorm
import matplotlib.pyplot as plt
import scipy.linalg as la

In [3]:
%%R
library(tidyverse)


UsageError: Cell magic `%%R` not found.


In [31]:
# Load Data
## Simulating Data
n = 1000
r = .95
inter = np.ones(n)
x1 = np.random.normal(0,5,n)
x2 = np.random.normal(0,2,n)
x34 = np.random.multivariate_normal([0, 0], np.array([[3, r], [r, 2]]), n)
X = np.c_[x1,x2,x34]
X = (X - X.mean(axis = 0)) / X.std(axis = 0)
X = np.c_[inter,X]

beta = np.array([1,-2,3,-1,2]).reshape(5,1)
pr = 1 / (1 + np.exp(X.dot(beta)))


# Possibly need to standardize this
y = np.random.binomial(n=1, p=pr)

In [32]:
np.c_[pr,y][:10,:]


array([[0.99541296, 1.        ],
       [0.55931515, 0.        ],
       [0.99031203, 1.        ],
       [0.99095291, 1.        ],
       [0.64595933, 1.        ],
       [0.11408075, 0.        ],
       [0.06377019, 0.        ],
       [0.94527421, 1.        ],
       [0.82094257, 1.        ],
       [0.81209617, 1.        ]])

In [27]:
n,d = X.shape

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in true_divide
  This is separate from the ipykernel package so we can avoid doing imports until


In [57]:
def log_normal_pdf(x,w,alpha):
    return(np.exp(-(np.log(x) - w)**2 / (2 * alpha**2)) / (x * alpha * np.sqrt(2 * np.pi)))

In [ ]:
def logistic_rmhmc(y,X,niter,burnin,leap_frog_steps= 6,num_newton_steps = 6):
    n,d = X.shape
    step_size = 3/leap_frog_steps
    w = np.ones(d)*(1e-3)
    wSaved = np.zeros(niter - burnin,d)
    # Set up a dictionary for the gradients
    g_deriv = {}
    invGdG = {}
    trInvGdG = np.zeros(d)
    
    log_prior = sum(np.zeros(d),w,alpha)
    f = X.dot(log_prior)
    loglike = f.T.dot(y) - np.sum(np.log(1 + np.exp(f)))
    current = loglike + log_prior
    prop = 0
    
    for i in range(niter):
        wNew = w
        prop += 1
        
        # New G
        f = X.dot(wNew)
        p = 1 / (1+ np.exp(-f))
        v = p.dot(1 - p)
        G = X.T.dot(v.dot(X)) + np.identity(d)/alpha
        
        invG = la.inv(G)
        
        oG = G
        o_chol_G = la.cholesky(G)
        o_inv_G = invG
        
        for j in range(d):
            Z = ((1 - 2*p).dot(X[:,d])) # Review
            g_deriv[j] = X.T.dot(v.dot(np.diag(Z))).dot(X)
            invGdG[j] = InvG.dot(g_deriv[j])
            trInvGdG[j] = np.trace(invGdG[j])
            
        p_momentum = np.random.randn(p).dot(oG)
        o_momentum = p_momentum
        
        if np.random.randn(1) > .5:
            TimeStep = 1
        else:
            TimeStep = -1
            
        RandomSteps = np.ceil(np.random.rand(1)*leap_frog_steps)
        
        SavedSteps = wNew
        
        # Leapfrog steps
        for step in range(RandomSteps):
            
            # Updating the Momentum
            dLdTheta = (X.T.dot(y - (1/(1 + np.exp(-f))))) - np.identity(d)*(1 / alpha).dot(wNew)
            trace1 = trInvGdG/2
            
            # Multiple fixed point iteration
            pm = p_momentum
            for fixed_i in range(num_newton_steps):
                Momentum

In [51]:
np.random.randn(3, 4)

array([ 0.51301974, -1.93166497,  0.53002304])

In [50]:
test_d = {}
test_d[0] = np.array([1,2,3,4,5,6]).reshape(2,3)
test_d

{0: array([[1, 2, 3],
        [4, 5, 6]])}

In [45]:
np.diag([1,2,3,4,5])

array([[1, 0, 0, 0, 0],
       [0, 2, 0, 0, 0],
       [0, 0, 3, 0, 0],
       [0, 0, 0, 4, 0],
       [0, 0, 0, 0, 5]])

In [36]:
test_x = np.random.multivariate_normal([0, 0], np.array([[3, r], [r, 2]]),2)
test_x
la.inv(test_x).dot(test_x)

array([[1.00000000e+00, 3.20499729e-17],
       [1.70818855e-16, 1.00000000e+00]])

In [43]:
A = np.array([
    [1,3,5],
    [3,13,23],
    [5,23,42]
])
print(la.cholesky(A))
print(la.inv(A))
print(la.inv(A).dot(A))

[[1. 3. 5.]
 [0. 2. 4.]
 [0. 0. 1.]]
[[ 4.25 -2.75  1.  ]
 [-2.75  4.25 -2.  ]
 [ 1.   -2.    1.  ]]
[[ 1.00000000e+00  3.77475828e-15  1.33226763e-15]
 [-4.44089210e-16  1.00000000e+00  1.33226763e-14]
 [ 9.99200722e-16 -5.55111512e-16  1.00000000e+00]]


In [28]:
X.std(axis = 0)

array([nan,  1.,  1.,  1.,  1.])

In [107]:
y = y
X = X
niter = 10
burnin = 2
leap_frog_steps = 6
num_newton_steps = 6
alpha = 100


n,d = X.shape
step_size = 3/leap_frog_steps
w = np.ones(d)*(1e-3)
wSaved = np.zeros((niter - burnin,d))
# Set up a dictionary for the gradients
g_deriv = {}
invGdG = {}
trInvGdG = np.zeros(d)
    
log_prior = sum(log_normal_pdf(np.zeros(d),w,alpha))
f = X.dot(log_prior)
loglike = f.T.dot(y) - np.sum(np.log(1 + np.exp(f)))
current = loglike + log_prior
prop = 0
    
for i in range(niter):
    wNew = w
    prop += 1
        
        # New G
    f = X.dot(wNew)
    p = 1 / (1+ np.exp(-f))
    v = p.dot(1 - p)
    G = X.T.dot(v*(X)) + np.identity(d)/alpha
        
    invG = la.inv(G)
        
    oG = G
    o_chol_G = la.cholesky(G)
    o_inv_G = invG
        
    for j in range(d):
        Z = ((1 - 2*p)*(X[:,j])) # Review
        g_deriv[j] = X.T.dot(v*(np.diag(Z))).dot(X) # My v is a scalar, but should it be?
        invGdG[j] = invG.dot(g_deriv[j])
        trInvGdG[j] = np.trace(invGdG[j])
            
    p_momentum = np.random.randn(d).dot(oG)
    o_momentum = p_momentum
        
    if np.random.randn(1) > .5:
        TimeStep = 1
    else:
        TimeStep = -1
            
    RandomSteps = np.ceil(np.random.rand(1).item()*leap_frog_steps)
        
    SavedSteps = wNew
        
        # Leapfrog steps
    for step in np.arange(RandomSteps):
            
            # Updating the Momentum
        dLdTheta = (X.T.dot(y - (1/(1 + np.exp(-f))))) - (np.identity(d)*(1 / alpha)).dot(wNew)
        # The above line doesn't broad cast 5,1000 and (5,).  Is this on my end or what?
        trace1 = trInvGdG/2
            
            # Multiple fixed point iteration
        pm = p_momentum
        last_term = np.zeros(d)
        MomentumHist = np.zeros((num_newton_steps,d))
        for fixed_i in range(num_newton_steps):
            #Momentum
            MomentumHist[fixed_i,:] = pm
            
            invGMomentum = invG.dot(pm)
            for dim in range(d):
                last_term[dim] = .5*(pm.dot(invGdG[dim].dot(invGMomentum)))
            
            pm = p_momentum + TimeStep * (step_size / 2) * (dLdTheta - trace1 + last_term.T)
        p_momentum = pm
        # Update w parameters
        oInvGMomentum = la.inv(G).dot(p_momentum) # They use a matlab code \
        # This is either inv(G).dot(p_momentum)
        
        
        pw = wNew
        wHist = np.zeros((num_newton_steps,d))
        for fixed_i in range(num_newton_steps):
            wHist[fixed_i,:] = pw
            
            f = X.dot(pw)
            p = 1 / (1 + np.exp(-f))
            
            v = p *(1 - p)
            G = (X.T * np.tile(v.T,(d,1))).dot(X) + (np.identity(d) / alpha)
            invGMomentum = la.inv(G).dot(p_momenum)
            
            pw = wNew + (Timestep * (step_size/2))*oInvGMomentum + (TimeStep*(step_size/2))*invGMomentum
            
        wNew = pw
        
        # Calculate G based on new parameters
        f = X.dot(wNew)
        p = 1 / (1+ np.exp(-f))
        

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: RuntimeWarning: divide by zero encountered in log
  
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in true_divide
  


ValueError: operands could not be broadcast together with shapes (5,1000) (5,) 

In [102]:
# Actual Analysis
type(v)
list(range(d))

test1 = np.array([1,2,3,4,5])
test2 = np.array([1,2,3,4,5])
test1 *test2

oG.shape
np.random.randn(d)
test = np.random.rand(1)
print(test)
print(leap_frog_steps)

np.ceil(test*leap_frog_steps).item()
np.arange(np.ceil(np.random.rand(1).item()*leap_frog_steps))


[0.2055339]
6


array([0., 1.])

In [123]:
#(X.T.dot(y - (1/(1 + np.exp(-f))))) - np.eye(d)*(1 / alpha).dot(wNew)
test_x = np.array([1,2,3,4,5])
np.tile(test_x,(5,1))
np.identity(5)

array([[1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1.]])